In [6]:
#Stockdata crawler (1mins)

import random 
from google.colab import drive
import pandas as pd
import glob
import os 
import datetime
import requests  

API_KEY_ARR = ["WCW741MHOAGYHV7J","9EOM89VBWKIGF0VU","TXXTM13PFRNXBC8S"]

def downloadFile(ticker,path,yrIndex,monthIndex,file_url):
  print("downloading {0}".format(file_url))
  temp_path = '{0}/{1}'.format(path,str(datetime.date.today()))
  if not os.path.exists(temp_path):
    os.makedirs(temp_path)  
  r = requests.get(file_url, stream = True)
  with open("{0}/{1}/yr{2}m{3}.csv".format(path,str(datetime.date.today()),str(yrIndex),str(monthIndex)) , "wb") as file:  
    for block in r.iter_content(): 
      if block:
        file.write(block)

def checkingResultFileExist(ticker): 
  root_path = '/content/drive/MyDrive/stockResearch'
  dataset_path = root_path + '/dataset/{0}/result/{0}_result.csv'.format(ticker)
  return os.path.exists(dataset_path)

def fileExistHandler(ticker):
  root_path = '/content/drive/MyDrive/stockResearch'
  dataset_path = root_path + '/dataset/{0}'.format(ticker)
  result_path = root_path + '/dataset/{0}/result/{0}_result.csv'.format(ticker)

  tempdf = pd.read_csv(result_path, index_col=None, header=0)
  tempdf = tempdf.sort_values(by='time',ascending=False)
  lastDateOfData = tempdf['time'].values[0]
  a = datetime.datetime.strptime(lastDateOfData, '%Y-%m-%d %H:%M:%S').date()
  b = datetime.date.today()
  for monthIndex in range(1,2 if ((b-a).days)//30 == 0 else (((b-a).days)//30)+1):
    file_url = "https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol=" + str(ticker) +"&interval=1min&slice=year1month"+ str(monthIndex) + "&adjusted=true&apikey=" + str(random.choice(API_KEY_ARR))
    downloadFile(ticker,dataset_path,1,monthIndex,file_url)
  tempPath = dataset_path+"/"+str(datetime.date.today())
  all_files = glob.glob(tempPath + "/*.csv".format())
  li = []
  for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)
  result = (pd.concat([tempdf, df]).drop_duplicates()).sort_values(by='time',ascending=False)
  result.to_csv('{0}/result/{1}_result({2}).csv'.format(dataset_path,ticker,str(datetime.date.today()), index=False)) 
  print('Data Updated: {0}/result/{1}_result({2}).csv Created'.format(dataset_path,ticker,str(datetime.date.today())))

def scrap(ticker):
  if checkingResultFileExist(ticker):
    fileExistHandler(ticker)
  else:
    root_path = '/content/drive/MyDrive/stockResearch'
    dataset_path = root_path + '/dataset/{0}'.format(ticker)
    tempPath = dataset_path+"/"+str(datetime.date.today())
    if not os.path.exists(dataset_path):
      os.makedirs(dataset_path)
    for yrIndex in range(1,3):
      for monthIndex in range(1,13):
        file_url = "https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol=" + str(ticker) +"&interval=1min&slice=year" + str(yrIndex) +"month"+ str(monthIndex) + "&adjusted=true&apikey=" + str(random.choice(API_KEY_ARR))
        print("downloading {0}".format(file_url))
        downloadFile(ticker,dataset_path,yrIndex,monthIndex,file_url)
 
    all_files = glob.glob(dataset_path + "/*.csv".format(ticker))
    li = []
    for filename in all_files:
      df = pd.read_csv(filename, index_col=None, header=0)
      li.append(df)
    frame = pd.concat(li, axis=0, ignore_index=True)
    frame = frame.sort_values(by='time',ascending=False)
    if not os.path.exists(dataset_path+'/result'):
      os.makedirs(dataset_path+'/result')
    frame.to_csv('{0}/result/{1}_result.csv'.format(dataset_path,ticker), index=False) 
    print('Data Created: {0}/result/{1}_result.csv Created'.format(dataset_path,ticker))


In [7]:
ticker = ["AAPL"]

for tickerItem in ticker:
  print(tickerItem)
  scrap(tickerItem)

AAPL
downloading https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol=AAPL&interval=1min&slice=year1month1&adjusted=true&apikey=TXXTM13PFRNXBC8S
